In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('train.csv')
data

,ID,text,sentiment
0,21098,".с.,и спросил его: о Посланник Аллаха!Ты пори...",1
1,21099,Роднее всех родных Попала я в ГКБ №8 еще в дек...,1
2,21100,Непорядочное отношение к своим работникам Рабо...,2
3,21101,"). Отсутствуют нормативы, Госты и прочее, что ...",1
4,21102,У меня машина в руках 5 лет и это п...,1
...,...,...,...
189886,210984,"Мой юбилей я отмечал в ресторане "" Астория "" ....",2
189887,210985,"Отлично встретили, разместили в роскошном номе...",1
189888,210986,Была в Васаби на ст. метро Сенная . Во первых...,0
189889,210987,Ребята не стоит смотреть этот фильм. Вы молоды...,0


In [1]:
!pip install -U lightautoml

  Using cached lightautoml-0.3.8.1-py3-none-any.whl.metadata (16 kB)
  Using cached AutoWoE-1.3.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached catboost-1.2.3-cp311-cp311-macosx_11_0_universal2.whl.metadata (1.2 kB)
  Using cached cmaes-0.10.0-py3-none-any.whl.metadata (19 kB)
  Using cached holidays-0.45-py3-none-any.whl.metadata (22 kB)
  Using cached joblib-1.2.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached json2html-1.3.0-py3-none-any.whl
  Using cached lightgbm-3.2.1.tar.gz (1.5 MB)
  Preparing metadata (setup.py) ... done
  Using cached optuna-3.6.0-py3-none-any.whl.metadata (17 kB)
  Using cached poetry_core-1.9.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached statsmodels-0.14.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.0 kB)
  Using cached torch-2.0.0-cp311-none-macosx_11_0_arm64.whl.metadata (23 kB)
  Using cached StrEnum-0.4.15-py3-none-any.whl.metadata (5.3 kB)
  Using cached pytest-8.1.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached sphinx-7.2.6-py3-none

In [3]:
np.random.seed(42)

train, test = train_test_split(data.iloc[:100000], test_size=0.25, random_state=42)

In [ ]:
!pip install lightgbm

  Using cached lightgbm-4.3.0.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done

In [4]:
from lightautoml.automl.presets.text_presets import TabularNLPAutoML
from lightautoml.tasks import Task

In [5]:
roles = {
    'text': ['text'],
    'drop': ['ID'],
    'target': 'sentiment',
}

task = Task('multiclass')

In [6]:
automl = TabularNLPAutoML(
    task=task,
    cpu_limit=1, gpu_ids='0',
    text_params = {'lang': 'ru', 'bert_model': 'DeepPavlov/rubert-base-cased-conversational'},
    autonlp_params={
        'model_name': 'random_lstm',
        'transformer_params': {
            'model_params': {
                'embed_size': 300,
                'hidden_size': 256,
                'pooling': 'mean',
                'num_layers': 2,
            },
            'dataset_params': {'embed_size': 300},
        },
    },
    nn_params={
        'max_length': 256,
        'bs': 1,
        'n_epochs': 10,
    },
)

In [7]:
oof_pred = automl.fit_predict(train, roles=roles, verbose=10)
test_pred = automl.predict(test)
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)

[12:25:15] Stdout logging level is DEBUG.


INFO:lightautoml.automl.presets.base:Stdout logging level is DEBUG.


[12:25:15] Model language mode: ru


INFO3:lightautoml.automl.presets.text_presets:Model language mode: ru


[12:25:15] Task: multiclass



...

INFO:lightautoml.automl.presets.base:Automl preset training completed in 168.85 seconds



[12:28:04] Model description:
Final prediction for new objects (level 0) = 
	 0.92500 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.07500 * (3 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 0.92500 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.07500 * (3 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) 



[12:28:26] Feature concated__text transformed


INFO3:lightautoml.transformers.text:Feature concated__text transformed


In [48]:
pred1 = []
for i in oof_pred:
    pred1.append(np.argmax(i.data))

pred2 = []
for i in test_pred:
    pred2.append(np.argmax(i.data))

In [ ]:
oof_pred = np.array

In [53]:
from sklearn.metrics import f1_score

In [59]:
print('Check scores:')
print('OOF score: {}'.format(f1_score(train[roles['target']].values[not_nan], np.array(pred1), average=None)))
print('TEST score: {}'.format(f1_score(test[roles['target']].values, np.array(pred2), average=None)))

Check scores:
OOF score: [0.52470588 0.7332959  0.65822785]
TEST score: [0.57978723 0.75128205 0.63662791]


In [55]:
train[roles['target']].values[not_nan]

array([1, 2, 1, ..., 0, 1, 2])

In [24]:
test = pd.read_csv('test.csv')
test

,ID,text
0,0,Развода на деньги нет Наблюдаюсь в Лайфклиник ...
1,1,Отель выбрали потому что рядом со стадионом. О...
2,2,"Вылечили Гноился с рождения глазик, в поликлин..."
3,3,Хорошее расположение.С вокзала дошли пешком.Но...
4,4,"Отличное месторасположение,прекрасный вид,особ..."
...,...,...
21093,21093,Несколько лет назад муж останавливался в этом ...
21094,21094,Спасли от боли После родов у меня появились бо...
21095,21095,з ролика понятно одно - девушка- наблюдатель ...
21096,21096,"Остались всем довольны, дружелюбный персонал, ..."


In [10]:
pred1 = automl.predict(test.iloc[:5000]).data
pred2 = automl.predict(test.iloc[5000:10000]).data
pred3 = automl.predict(test.iloc[10000:15000]).data
pred4 = automl.predict(test.iloc[15000:20000]).data
pred5 = automl.predict(test.iloc[20000:]).data

[12:29:57] Feature concated__text transformed


INFO3:lightautoml.transformers.text:Feature concated__text transformed


[12:30:53] Feature concated__text transformed


INFO3:lightautoml.transformers.text:Feature concated__text transformed


[12:31:34] Feature concated__text transformed


INFO3:lightautoml.transformers.text:Feature concated__text transformed


[12:32:27] Feature concated__text transformed


INFO3:lightautoml.transformers.text:Feature concated__text transformed


[12:32:38] Feature concated__text transformed


INFO3:lightautoml.transformers.text:Feature concated__text transformed


In [14]:
res = []

In [21]:
for i in pred1:
    res.append(np.argmax(i.data))
for i in pred2:
    res.append(np.argmax(i.data))
for i in pred3:
   res.append(np.argmax(i.data))
for i in pred4:
    res.append(np.argmax(i.data))
for i in pred5:
    res.append(np.argmax(i.data))

In [29]:
pd.DataFrame(pd.Series(np.array(res))).to_csv('submission.csv', sep=',', index=True, encoding='utf-8')

In [ ]:
import joblib
joblib.dump(automl,'models/automl_from_chris.pkl')